In [2]:
from langchain_pinecone import vectorstores
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

openai_embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))

# hugging_face = HuggingFaceHubEmbeddings(huggingfacehub_api_token=os.getenv("HUGGING_FACE_API_KEY"))
pc = vectorstores.Pinecone(
    pinecone_api_key=os.getenv("PINECONE_API_KEY"),
    embedding=openai_embeddings,
    index_name="finalindex",
)


# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"), temperature=0.5
)

/home/rohanbendapudi/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/home/rohanbendapudi/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.3 and will be removed in 0.3.0. Use PineconeVectorStore instead.
  warn_deprecated(


In [17]:
from langchain_community.llms.ollama import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub

# Load the system prompt
with open("system_prompt.txt", "r") as file:
    system_prompt = file.read().replace("\n", "")

# Get human input
human = input("Enter your question: ")

# Define the template
template = f"{system_prompt}\n\nQuestion: {human}"

# Create the prompt template
prompt = ChatPromptTemplate.from_template(template=template)

# Define the RAG chain
rag_chain = (
    {
        "context": pc.as_retriever(k=2, search_type="similarity"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Invoke the chain
result = rag_chain.invoke(human)

# Print the result
print(result)

The primary keys of the PJT_INT_ATTRIBUTE_MAPPING table in Oracle Fusion Project Management are:

1. ATTRIBUTE_MAPPING_ID: This is a unique identifier for each attribute mapping record in the table.
2. PROJECT_ID: This key links the attribute mapping to a specific project within the system.

These primary keys are essential for maintaining data integrity and ensuring accurate mapping of attributes within Oracle Fusion Project Management. If you need more specific details or guidance on working with the PJT_INT_ATTRIBUTE_MAPPING table, feel free to ask for further assistance.


In [14]:
pc.similarity_search(query="What are the primary keys of PJE_ISSUES_TL?", k=1)

[Document(page_content=' PJE_ISSUES_TL                \t\tTables and Views for Project ManagementPJE_ISSUES_TLPJE_ISSUES_TL   This is a MLS table for PJE_ISSUES_B Details   Schema: FUSION   Object owner: PJE   Object type: TABLE   Tablespace: FUSION_TS_TX_DATA    Primary Key   Name  Columns     PJE_ISSUES_TL_PK   ISSUE_ID, LANGUAGE, ENTERPRISE_ID     Columns   Name Datatype Length Precision Not-null Comments    ENTERPRISE_ID NUMBER  18 Yes Column to support multitenancy   ISSUE_ID NUMBER  18 Yes The Unique identifier of the Issue ,uses a global sequence ,refers PJE_ISSUES_B.ISSUE_ID   LANGUAGE VARCHAR2 4  Yes Indicates the code of the language into which the contents of the translatable columns are translated.   SOURCE_LANG VARCHAR2 4  Yes Indicates the code of the language in which the contents of the translatable columns were originally created.   CREATED_BY VARCHAR2 64  Yes Who column: indicates the user who created the row.   CREATION_DATE TIMESTAMP   Yes Who column: indicates the 

In [19]:
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import HumanMessagePromptTemplate


with open("system_prompt.txt", "r") as file:
    system_prompt = file.read().replace("\n", "")

retriever = pc.as_retriever(k=3, search_type="similarity")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "{context}"),
        ("human", "{input}"),
        (
            "ai",
            "Here, I can give you details on the table, and a SQL query to get the data",
        ),
    ]
)
print(prompt)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"context": system_prompt, "input": input()})
print(response["answer"])

input_variables=['context', 'input'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='{context}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Here, I can give you details on the table, and a SQL query to get the data'))]
The columns of the `PJT_PLAN_VERSIONS` table are as follows:

1. `PLAN_VERSION_ID`: System-generated identifier for the project plan version
2. `PLAN_STATUS_CODE`: Indicates whether it is a "baselined", "published" or "working" version
3. `PROJECT_ID`: Project for which the plan version is created
4. `PUBLISHED_BY_PERSON_ID`: Resource Id of a person who published the project plan
5. `PUBLISHED_DATE`: Date the most recent project plan version was published
6. `OBJECT_VERSION_NUMBER`: Used to implement optimistic locking
7. `CREATED_BY`: Who column: indicates the user who created the row
8. `

In [31]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

retriever = pc.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = system_prompt

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)



while True:
    query = input("Ask a question, type 'exit' to finish: \n")

    if query == "exit":
        print("Goodbye!!!")
        break
    else:
        answer = conversational_rag_chain.invoke(
            {"input": query, "context": system_prompt}, config={"configurable": {"session_id": "abc123"}}
        )["answer"]
        print(answer + "\n")

Parent run e52f37f7-179f-47a1-ad23-7df25cb0d4a3 not found for run 3b08b4f1-8836-4a68-9c1d-7d0386148b34. Treating as a root run.


The primary key of PJT_PLAN_VERSIONS is PLAN_VERSION_ID.



Parent run c4fe0fa7-7f09-4233-b87a-7b9171570a70 not found for run 019838b5-3faf-4086-8d95-a21fd340d88d. Treating as a root run.


The columns of PJT_PLAN_VERSIONS are:
1. PLAN_VERSION_ID
2. PLAN_STATUS_CODE
3. PROJECT_ID
4. PUBLISHED_BY_PERSON_ID
5. PUBLISHED_DATE
6. OBJECT_VERSION_NUMBER
7. CREATED_BY
8. CREATION_DATE
9. LAST_UPDATED_BY
10. LAST_UPDATE_DATE
11. LAST_UPDATE_LOGIN
12. PARENT_STRUCTURE_VERSION_ID
13. ENTERPRISE_ID
14. LAST_SCHEDULE_DATE
15. FIN_DATES_REQUEST_ID
16. FIN_DATES_ESS_DATE
17. FIN_DATES_ESS_STATUS
18. FIN_DATES_ESS_MSG_CODE



Parent run 12eb0482-bc2d-4cd3-ab6e-72f96c685a41 not found for run 0909c8b4-9956-4126-9e5f-25657cc056ad. Treating as a root run.


I see that you haven't provided a question or follow-up. If you have any more questions or need further assistance, feel free to ask!

Goodbye!!!
